<a href="https://colab.research.google.com/github/pedrobslima/ultrawiki-mining/blob/main/link_mining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Imports

In [ ]:
%%capture
!pip install python-igraph
!pip install pyvis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 12.7 MB/s eta 0:00:00


In [299]:
import pandas as pd
from pandas import DataFrame, Series
import numpy as np
#import scrapy
import requests
from http.client import responses
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import networkx as nx
from pyvis.network import Network
import igraph
from IPython.core.display import display, HTML
#import seaborn as sns

### Main

In [ ]:
main_page_url = "https://ultrakill.wiki.gg/wiki/Main_Page"
domain_url = "https://ultrakill.wiki.gg"
base_url = 'https://ultrakill.wiki.gg/wiki/'

In [288]:
df = DataFrame(columns=['from', 'to']) # DataFrame({'from': ['Main_Page'], 'to':['aaaa']})

In [284]:
def link_scraping(url:str):
    global df
    if(url in df['from'].values):
        print(f'[{len(df)}] Tentativa de loop: ->{url[31:]}')
    else:
        # Fazer uma solicitação
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')

        # Criar top_items como uma lista vazia
        #all_links = []

        # Extrair e armazenar em top_items de acordo com as instruções
        links = soup.select('main a')
        for ahref in links:
            href = ahref.get('href')
            href = href.strip() if href is not None else ''
            while('#' in href):
                href = href[:href.rfind('#')]
            name = href[6:]

            # tvz desconsiderar a parte dps da #, como em 'Sawblade_Launcher#Attractor-0'
            if((href[:6] == '/wiki/') and not(('File:' in name) or ('Special:' in name) or ('ULTRAKILL_Wiki:' in name) or ('Category:' in name) or ('Template:' in name) or ('User:' in name))):
                href = href[:href.rfind(':')] if ':' in name else href
                new_url = domain_url+href
                df = pd.concat([df, DataFrame({'from':url, 'to':new_url}, index=[len(df)])])
                link_scraping(new_url)
                #all_links.append({'name': href[6:], "href": domain_url+href})

        #print(all_links)
        #print(len(all_links))

In [289]:
link_scraping(main_page_url)
dfc = df.copy()

A saída de streaming foi truncada nas últimas 5000 linhas.
[25998] Tentativa de loop: ->Gutterman
[25999] Tentativa de loop: ->Guttertank
[26000] Tentativa de loop: ->Guttertank
[26001] Tentativa de loop: ->V2
[26002] Tentativa de loop: ->V2
[26003] Tentativa de loop: ->V2
[26004] Tentativa de loop: ->V2
[26005] Tentativa de loop: ->1000-THR_%22Earthmover%22
[26006] Tentativa de loop: ->1000-THR_%22Earthmover%22
[26007] Tentativa de loop: ->Malicious_Face
[26008] Tentativa de loop: ->Malicious_Face
[26009] Tentativa de loop: ->Cerberus
[26010] Tentativa de loop: ->Cerberus
[26011] Tentativa de loop: ->Idol
[26012] Tentativa de loop: ->Idol
[26013] Tentativa de loop: ->Mannequin
[26014] Tentativa de loop: ->Mannequin
[26015] Tentativa de loop: ->Hideous_Mass
[26016] Tentativa de loop: ->Hideous_Mass
[26017] Tentativa de loop: ->Leviathan
[26018] Tentativa de loop: ->Leviathan
[26019] Tentativa de loop: ->Minotaur
[26020] Tentativa de loop: ->Minotaur
[26021] Tentativa de loop: ->Virtue


In [267]:
#df = dfc.copy()
#df

,from,to
0,https://ultrakill.wiki.gg/wiki/Main_Page,https://ultrakill.wiki.gg/wiki/Weapons
1,https://ultrakill.wiki.gg/wiki/Weapons,https://ultrakill.wiki.gg/wiki/Spawner_Arm
2,https://ultrakill.wiki.gg/wiki/Spawner_Arm,https://ultrakill.wiki.gg/wiki/Sandbox
3,https://ultrakill.wiki.gg/wiki/Sandbox,https://ultrakill.wiki.gg/wiki/Cheats
4,https://ultrakill.wiki.gg/wiki/Cheats,https://ultrakill.wiki.gg/wiki/Sandbox
...,...,...
39423,https://ultrakill.wiki.gg/wiki/Main_Page,https://ultrakill.wiki.gg/wiki/Soundtrack
39424,https://ultrakill.wiki.gg/wiki/Main_Page,https://ultrakill.wiki.gg/wiki/Soundtrack
39425,https://ultrakill.wiki.gg/wiki/Main_Page,https://ultrakill.wiki.gg/wiki/Development
39426,https://ultrakill.wiki.gg/wiki/Development,https://ultrakill.wiki.gg/wiki/Developer_Museum


In [290]:
df['to'].value_counts()

to
https://ultrakill.wiki.gg/wiki/Malicious_Face             823
https://ultrakill.wiki.gg/wiki/Spawner_Arm                680
https://ultrakill.wiki.gg/wiki/Shotgun                    615
https://ultrakill.wiki.gg/wiki/Impact_Hammer              614
https://ultrakill.wiki.gg/wiki/Filth                      610
                                                         ... 
https://ultrakill.wiki.gg/wiki/Weapon                       1
https://ultrakill.wiki.gg/wiki/Level                        1
https://ultrakill.wiki.gg/wiki/1000-THR_Defence_System      1
https://ultrakill.wiki.gg/wiki/Sisyphus                     1
https://ultrakill.wiki.gg/wiki/Difficulty                   1
Name: count, Length: 221, dtype: int64

In [291]:
# formatanto strings
df['from'] = df['from'].str.replace(base_url, '')
df['to'] = df['to'].str.replace(base_url, '')

# removendo loops diretos
df = df[df['from']!=df['to']]

# ordenando strings por linha
df = DataFrame(np.sort(df.values, axis=1), columns=df.columns)

# agrupando conexões
df['weight'] = 1
df = df.groupby(['from', 'to'], sort=False, as_index=False).sum()
df

,from,to,weight
0,Main_Page,Weapons,2
1,Spawner_Arm,Weapons,2
2,Sandbox,Spawner_Arm,2
3,Cheats,Sandbox,2
4,Cheats,Hookpoint,1
...,...,...,...
7989,Lore,Main_Page,2
7990,Enemies,Main_Page,2
7991,Main_Page,Soundtrack,2
7992,Development,Main_Page,2


In [292]:
df.sort_values('weight', ascending=False)

,from,to,weight
7595,Level,Soul_Orbs,102
2159,Levels,Soul_Orbs,102
4206,Final_War,V1,58
17,Lore,V1,58
4030,Long_Night,V1,57
...,...,...,...
6524,Coins,Sawblade_Launcher,1
6523,Coins,Nailgun,1
2805,Nailgun_(disambiguation),Railcannon,1
6511,Coins,Weapons,1
